<a href="https://colab.research.google.com/github/bforoura/Transformers/blob/main/nlp_transformers_ch6_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Step 1: Activate the GPU

# Go to Runtime / Change runtime type / Hardware Accelerator GPU



In [1]:
#@title Step 2: Cloning the OpenAI GPT-2 Repository

!git clone https://github.com/openai/gpt-2.git


Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 4.51 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [3]:
#@title Step 3: Installing the requirements


# import os          # when the VM restarts import os necessary
# os.chdir("/content/gpt-2")
# !pip3 install -r requirements.txt

# The requirements have been installed automatically.

!pip uninstall tensorflow
!pip install tensorflow==1.15.2




Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.9.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 38 kB/s 
     |████████████████████████████████| 503 kB 67.6 MB/s 
     |████████████████████████████████| 3.8 MB 42.3 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2e8f933c896fafc55c1707fa2b

In [4]:
#@title Step 4: Checking the Version of TensorFlow

#Colab includes Tensorflow 2.x by default but we need a lower version

import tensorflow as tf

print(tf.__version__)



1.15.2


In [6]:
#@title Step 5: Downloading the 345M parameter GPT-2 Model

# run code and send argument
import os 

os.chdir("/content/gpt-2")
!python3 download_model.py '345M'


Fetching checkpoint: 1.00kit [00:00, 1.00Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:01, 559kit/s]                                                    
Fetching hparams.json: 1.00kit [00:00, 888kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [03:52, 6.10Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 8.18Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:01, 483kit/s]                                                  
Fetching vocab.bpe: 457kit [00:01, 360kit/s]                                                        


In [7]:
#@title Step 6: Printing UTF encoded text to the console

!export PYTHONIOENCODING=UTF-8


In [8]:
#@title Step 7: Project Source Code


os.chdir("/content/gpt-2/src")



In [9]:
#@title Step 7a: Interactive Conditional Samples (src)

#Project Source Code for Interactive Conditional Samples:
# /content/gpt-2/src/interactive_conditional_samples.py file
import json
import os
import numpy as np
import tensorflow as tf



In [10]:
#@title Steps 7b-8: Importing and defining the model


import model, sample, encoder


In [12]:
#@title Step 8: Defining the model

def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
#@title Step 9: Interacting with GPT-2

interact_model('345M',None,1,1,300,1,0,'/content/gpt-2/models')






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.


Model prompt >>> Human reason, in one sphere of its cognition, is called upon to consider questions, which it cannot decline, as they are presented by its own nature, but which it cannot answer, as they transcend every faculty of the mind.
======================================== SAMPLE 1 ========================================
 Argumentum ad primum est reasoningis; which is nothing but, and therefore of no force at all, as Reason in general pejoratively defines it, still is an cardinal reason. And how was it to be done, otherwise than this, that Reason in every sphere of its cognition ought to select one particular particular serene opinion, and admit no pervading persuasion or evidence from outside of that, but altogether repose itself in private no less fully, than it should do in common? Surely, indeed, constructivism was not ready according to any principles of its own; conceived from any principle superior to itself,—but rule and prerogative right, recognised 1***6 and both cons